## Prototypical GA applied to the TSP

This prototypical GA aims to solve the travelling salesman problem for fully connected graphs without self-connections where each node must be visited exactly once with no requirement to return to the source node. 

**COMMENT** 

**This variation of the crossover function produces duplicates in the children**

def crossover(h_a, h_b):
    x, y = tuple(np.array_split(h_a, 2))
    g, f = tuple(np.array_split(h_b, 2))
    
    return np.concatenate((x, f)), np.concatenate((g, y))

In [73]:
import random
import numpy as np
import networkx as nx

In [37]:
# Fitness function is the inverse of the pathlength 
def fit(h, g):
    return (1 / sum(g[i][j]['weight'] for i, j in zip(h, h[1:])))

In [3]:
# Define a population generation method 
# For the TSP every node must be visisted at least once 
# The nodes can be visited in any order without a specific source node 
def gen_population(size, g):
    nodes = list(g.nodes())
    return np.array([np.random.permutation(nodes) for x in range(size)])

In [4]:
# The prototypical mutation scheme is to invert a random bit from a gene
#              selected uniformly 
# In this case, we need sequences of nodes without duplicates 
# The mutation scheme here will be to swap the index of two nodes, chosen uniformly 
def mutate(h):
    # Uniformly select two values from np.arrange(0, len(h))
    a, b = tuple(np.random.choice(len(h), 2))
    # Swap these values 
    h[a], h[b] = h[b], h[a]
    
    return h 

In [11]:
def crossover(a, b):
    # Select two random indexes 
    x, y = np.random.choice(len(a), 2)
    
    if x > y: 
        x, y = y, x # Enforce x < y 
    
    # Create an empty array 
    child_a = [None] * len(a)
    child_b = [None] * len(a)
    
    # Slice 
    child_a[x:(y+1)] = a[x:(y+1)]
    child_b[x:(y+1)] = b[x:(y+1)]
    
    # Replace remaining elements of alternate parent
    ix = list(range(0, x)) + list(range((y + 1), len(a)))
    
    ind = 0
    for gene in b:
        if gene not in child_a:
            child_a[ix[ind]] = gene
            ind += 1 
    
    ind = 0 
    for gene in a:
        if gene not in child_b:
            child_b[ix[ind]] = gene
            ind +=1 

    return child_a, child_b

In [50]:
# p: size of population 
# r: fraction of population to be replaced by crossover 
# m: mutation rate 
# g; graph 
def ga(threshold, itr, p, r, m, g):
    # Generate a random population 
    population = gen_population(p,g)
    
    # Compute fitness for each hypothesis 
    fitness = [fit(h, g) for h in population]
    
    # Determine maximum fitness
    fit_max = max(fitness) 
    
    # Determine total fitness
    fit_total = sum(fitness)
    
    # Calculate maximum of fitness 
    count = 0
    while (fit_max < threshold) and (count < itr): 
        # Create new generation 
        # Select (1 - r)p hypotheses to persist 
        gen = []
        
        pr_sum = 0.0 
        while len(gen) < ((1 - r) * p):
            # Select a parent with probability
            # Pr(hi) = f(hi) / sum_j(f(hj))
            running_sum = 0.0
            rnd = random.random() 

            ind = 0 
            while running_sum < rnd: 
                running_sum += (fitness[ind] / fit_total)
                ind += 1 
            
            # Add the selected element 
            gen.append(population[(ind - 1)])
        
        # Select (rp)/2 pairs for crossover with Pr(hi)
        n_parents = int(r * p) 
     
        step = fit_total / n_parents
        start = random.uniform(0, step)
        pointers = [(start + (i * step)) for i in range(0, n_parents)]
        
        parents = []
        for ptr in pointers:
            ind = 0
            fsum = 0.0 
            
            while fsum < ptr: 
                fsum += fitness[ind]
                ind += 1 
            
            parents.append(population[ind - 1])
        
        # Crossover 
        pairs = zip(parents, parents[1:])
        for a, b in pairs:
            c, d = crossover(a, b)
            gen.append(c)
            gen.append(d)
        
        # Choose m percent of new population to mutate 
        m_size = int(m * p)
        for i in np.random.choice(len(gen), m_size):
            gen[i] = mutate(gen[i])
        
        # Evaluate generation fitness 
        fitness = [fit(h, g) for h in gen]
        fit_max = max(fitness)
        
        # Population equal to copy of generation 
        population = list(gen) 
        
        # Increment iteration count
        count += 1 
    
    # Return best
    best = np.argmax(fitness)
    return (fitness[best], population[best])

In [47]:
# Create a graph to test this 
# Complete graph is required to generate travelling salesman problem 
G = nx.complete_graph(16)


# Then also need to assign weights to edges 
# These will be uniform integers in range 1, 100
for e in G.edges():
    G[e[0]][e[1]]['weight'] = random.randrange(1, 100)

In [59]:
# Set threshold to the lowest value found by bad algorithm in ten tries 
threshold = (1 / 200)

# Run the algorithm for the travelling salesman problem on this graph 
r = 0.2 
m = 0.05
p = 1000

best_fit, solution = ga(threshold, 1000, p, r, m, G)

In [60]:
print(best_fit)

0.0028735632183908046


In [61]:
print(solution)

[3, 14, 0, 10, 13, 1, 12, 15, 11, 7, 5, 8, 9, 6, 2, 4]


In [62]:
print(sum(G[i][j]['weight'] for i, j in zip(solution, solution[1:])))

348


In [72]:
tsp = nx.approximation.traveling_salesman_problem
path = tsp(G, nodes = None, cycle = None)

print(sum(G[i][j]['weight'] for i, j in zip(path, path[1:])))

[3, 10, 12, 10, 0, 9, 2, 9, 8, 5, 6, 14, 1, 11, 15, 13, 15, 4, 15, 7]
88


From this code we get duplicates. Problem!

[ 9 10  7  1 13 14 11  3  3 15 14  1  2  6  5 10] Example of bad array 

i,j such that Hi = Hj where j = (i+1) ... i.e. edge from node to itself which does not exist. 

How do we design a better crossover operation? 

The children need to visit every node; this is a requirement. If they do not, there is a problem. 

